Task-4 : Bengali
Using syllable n-grams with TF-IDF(custom analyser)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

Pre-processing

In [ ]:
!pip install emot --upgrade
!pip install emoji
!pip install demoji
!pip install contractions

In [ ]:
!git clone https://github.com/libindic/syllabalizer.git
%cd syllabalizer
%run setup.py sdist

In [ ]:
!pip install dist/libindic-syllabifier*.tar.gz
!pip install libindic-utils

  Preparing metadata (setup.py) ... done
  Created wheel for libindic-syllabifier: filename=libindic_syllabifier-0.0.1.dev28-py2.py3-none-any.whl size=8121 sha256=c27ca509278b5c832fa9a08fe125298216f0fd82b28ea0c1033ca2c5564eceb8
  Stored in directory: /root/.cache/pip/wheels/d2/50/19/e2620530fc718493df9f5bca90d23db025cc864b9c7c0a98ef
  Created wheel for silpa_common: filename=silpa_common-0.3-py3-none-any.whl size=8466 sha256=b57458d4b0d9f7b458bf5575320088fa024530c4f854563420ce6367e48248a2
  Stored in directory: /root/.cache/pip/wheels/c0/72/43/0c779b79d708c78240beb3b0bb8f5ff3c2ab81c4e5271ea1aa
Successfully built libindic-syllabifier silpa_common


In [ ]:
from libindic.syllabifier import Syllabifier
instance = Syllabifier()

In [ ]:
# instance.syllabify('*.')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import  CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.metrics import precision_recall_fscore_support as score
from textblob import TextBlob
import contractions
import emot
import emoji
import demoji
import nltk.data
import numpy as np
import pandas as pd
import re
import time
import string
import nltk
nltk.download('omw-1.4')
pd.set_option("max_colwidth" ,120)
#wn = nltk.WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('stopwords')
#stopword = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

english_stopwords = stopwords.words("english")
with open('/content/drive/MyDrive/Hasoc 2023/Task 4/Bengali/bn.txt', encoding = 'utf-8') as f:
    ben_stopwords = f.readlines()
    for i in range(len(ben_stopwords)):
        ben_stopwords[i] = re.sub('\n','', ben_stopwords[i])
stopwords = english_stopwords + ben_stopwords
#english_stemmer = SnowballStemmer("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
len(stopwords)

298

In [ ]:
demoji.download_codes()
def emo(text):
  temp=emoji.demojize(text,delimiters=(" "," "))
  temp=temp.replace("_","  ")
  return temp

<ipython-input-18-e00aa54160fe>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
punct='“!"#$%&\'।()*+,-./:;<=>?@[\\]^_`{|}~”'

In [ ]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'',text)

In [ ]:
def clean_text(text):
    text = re.sub('@[a-zA-Z0-9]*', '',text)#remove @ - A Z
    text=contractions.fix(text,slang=True)
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    text= "".join([word.lower() for word in text if word not in punct])
    text = "".join([word for word in text if not word.isdigit()])#remove digit
    text = " ".join(word for word in text.split() if word not in stopwords)#this is for not splitting other languages but it is not removing emojis
    if len(text) == 0:
      text= '**********************'
    return text

In [ ]:
df1=pd.read_csv('/content/drive/MyDrive/Hasoc 2023/Task 4/Bengali/train_BE_AH_HASOC2023.csv')
df2=pd.read_csv('/content/drive/MyDrive/Hasoc 2023/Task 4/Bengali/test_BE_AH_HASOC2023.csv')

In [ ]:
df1.head()

,S. No.,text,task_1
0,1,তবে শুনলাম মমতা ব্যানার্জি কোটা পদ্ধতি তুলে দিয়েছে তাহলে ব্রাত্য বসুর মুখে কোটা কেন? এটা কথার কথা? তাহলে কোটা নিয়ে...,NOT
1,2,"প্রত্যেক দলের প্রত্যেক মন্ত্রী এমএলএ, এমপি, এদের একটা নিজস্ব কোটা আছে, সেই কোটায় তারা যাকে খুশি চাকরি দিতে পারেন, এ...",NOT
2,3,"বিশ্ব জুড়ে বুদ্ধিজীবিদের সঙ্গে সম্পর্ক থাকে বিশ্ববিদ্যালয়,সেমিনার, পাবলিশিং হাউস ও মিডিয়ার। বাংলায় দেখছি গরু চোরাক...",HOF
3,4,প্রতি বছর শত শত মানুষ হত্যা করছে সীমান্তে গণতান্ত্রিক আর আইনের শাসনের দেশ ভারতের সীমান্তরক্ষীরা। পৃথিবীর কোন সভ্য দে...,NOT
4,5,সমাজের সবথেকে দূষিত মানুষ এরাই খারাপ করে দেয় সমাজকে ছি ছি ছি,HOF


In [ ]:
df2.head()

,S. No.,text
0,1,তাতে তোর কী? বেশি চুলকুনি নারে সালা ?
1,2,অবশ্য চটি চাটা হতে হবে
2,3,"KKR এর প্রসঙ্গ তুললে শাহরুখের ছবি দেয়া কি বাধ্যতামূলক? শাহরুখ KKR এর সহ-মালিক হতে পারে, ক্রিকেটার নয়। ক্রিকেট এবং ক্..."
3,4,হ্যা হ্যা ভিখিরি বলতেই পাশের দেশের লোকের লাগছে। যার যা নাম ধরে ডাকলেই চিনতে পারে।
4,5,এটা নিজের মেয়ের নাম রাখবেন আর এই নাম ধরে ডাকবেন .....😁😅😆


In [ ]:
df1['clean_text'] = df1['text'].apply(lambda x:emo(x))
df1['clean_text'] = df1['clean_text'].apply(lambda x:remove_urls(x))
df1['clean_text'] = df1['clean_text'].apply(lambda x:clean_text(x))

In [ ]:
df2['clean_text'] = df2['text'].apply(lambda x:emo(x))
df2['clean_text'] = df2['clean_text'].apply(lambda x:remove_urls(x))
df2['clean_text'] = df2['clean_text'].apply(lambda x:clean_text(x))

In [ ]:
df1['clean_text'][0:10]

0    শুনলাম মমতা ব্যানার্জি কোটা পদ্ধতি তুলে দিয়েছে তাহলে ব্রাত্য বসুর মুখে কোটা এটা কথার কথা তাহলে কোটা নিয়ে প্রাক্তন ...
1    প্রত্যেক দলের প্রত্যেক মন্ত্রী এমএলএ এমপি এদের একটা নিজস্ব কোটা কোটায় তারা যাকে খুশি চাকরি দিতে পারেন এবং চিরকাল এন...
2    বিশ্ব জুড়ে বুদ্ধিজীবিদের সম্পর্ক থাকে বিশ্ববিদ্যালয়সেমিনার পাবলিশিং হাউস মিডিয়ার বাংলায় দেখছি গরু চোরাকারবারিদের স...
3    বছর শত শত মানুষ হত্যা করছে সীমান্তে গণতান্ত্রিক আইনের শাসনের দেশ ভারতের সীমান্তরক্ষীরা পৃথিবীর কোন সভ্য দেশের সীমান্...
4                                                                  সমাজের সবথেকে দূষিত মানুষ এরাই খারাপ দেয় সমাজকে ছি ছি ছি
5    যারা প্রেরণার কথার মানে বুঝতে পারেনি তাদের মানে বোঝার জন্য ওর যেতে হবে ঠাণ্ডা মস্তিস্ক নিয়ে ভালো বোঝাতে পারবে আমার ...
6                                                                         কন্ডম দিদি grinning face sweat grinning face sweat
7                                                                                                         বড় বড় কথা কোথায়


In [ ]:
df2['clean_text'][0:10]

0                                                                                              তাতে তোর কী চুলকুনি নারে সালা
1                                                                                                               চটি চাটা হবে
2    kkr প্রসঙ্গ তুললে শাহরুখের ছবি দেয়া বাধ্যতামূলক শাহরুখ kkr সহমালিক পারে ক্রিকেটার ক্রিকেট এবং ক্রিকেটারকে ভালবেসে আপ...
3                                               হ্যা হ্যা ভিখিরি বলতেই পাশের দেশের লোকের লাগছে যার নাম ধরে ডাকলেই চিনতে পারে
4            এটা নিজের মেয়ের নাম রাখবেন নাম ধরে ডাকবেন beaming face smiling eyes grinning face sweat grinning squinting face
5    পিগি চপস মানে বাংলায় ওইটাই বোঝায় face tears joy face tears joy face tears joy face tears joy face tears joy আমার ম...
6                                                                                                  মাছেলে দুজনকেই ভালো লাগছে
7                                                                         ইয়ে মাথায় ওঠেনি এগুলো সবই পরিকল্পনা বিদ্বেষ ছাড়ানো


Feature extraction using Syllable n-grams with custom analyser

In [ ]:
from sklearn.feature_extraction.text import  CountVectorizer, TfidfTransformer,TfidfVectorizer

In [ ]:
def syllable(word):
    try:
        sylbls = instance.syllabify(word)
    except:
        sylbls = list(word)
    return sylbls

In [ ]:
# generate word n-grams

def generate_N_grams(text, ngram=1):
    words = [word for word in text.split(" ")]  # if word not in set(stopwords.words('english'))]
    temp = zip(*[words[i:] for i in range(0, ngram)])
    ngrams = [' '.join(ngram) for ngram in temp]
    return ngrams

In [ ]:
import regex

def custom_analyzer(text):
    words = regex.findall(r'\w{2,}', text) # extract words of at least 2 letters
    for w in words:
        yield w

In [ ]:
# generate syllable n-grams by specifying the value of n, by default ngram=1

def generate_cN_grams(word, ngram=1):
    sylbs = [sylbs for sylbs in syllable(word)]  # if word not in set(stopwords.words('english'))]
    temp = zip(*[sylbs[i:] for i in range(0, ngram)])
    ngrams = [''.join(ngram) for ngram in temp]
    return ngrams

In [ ]:
# generate syllable n-grams in the range 'from' to 'to'. by default 'from'= 1

def ngrams(text, r_to, r_from=1):
    r_from = 1
    r_to = 3
    allngrams=[]
    for word in text.split():
        temp = []
        for i in range(r_from + r_to - 1):
            temp.extend(generate_cN_grams(word, i+1))
            allngrams.append(temp)
    n_grams = [item for sublist in allngrams for item in sublist]
  # merged = list(itertools.chain(allngrams))
    return n_grams

In [ ]:
def custom_analyzer(text):
    syngrams = ngrams(text,3)
    return syngrams

Splitting train set

In [ ]:
#split the dataset into train and test sets
X_train1, X_test1, y_train1, y_test1 = train_test_split(df1.clean_text, df1.task_1, test_size=0.25, random_state=42)

In [ ]:
X_train1.shape, X_test1.shape

((960,), (321,))

In [ ]:
tfidf_vectorizer = TfidfVectorizer(analyzer=custom_analyzer)
tfidf_vectorizer.fit_transform(X_train1)
xtrain_tfidf =  tfidf_vectorizer.transform(X_train1)
xtest_tfidf =  tfidf_vectorizer.transform(X_test1)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
svm_model = SVC(kernel='linear', class_weight='balanced')

In [ ]:
svm_model.fit(xtrain_tfidf, y_train1)

SVC(class_weight='balanced', kernel='linear')

In [ ]:
y_pred = svm_model.predict(xtest_tfidf)

In [ ]:
accuracy1 = accuracy_score(y_test1, y_pred)

print("Test Accuracy:", round(accuracy1*100, 4))

print("\n", classification_report(y_test1, y_pred))

Test Accuracy: 63.5514

               precision    recall  f1-score   support

         HOF       0.56      0.53      0.54       131
         NOT       0.69      0.71      0.70       190

    accuracy                           0.64       321
   macro avg       0.62      0.62      0.62       321
weighted avg       0.63      0.64      0.63       321



In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

clf1 = LogisticRegression(random_state=1)
clf2 = BernoulliNB()
clf3 = LinearSVC(penalty='l2',C=1.0)
clf4=  MultinomialNB()

eclf = VotingClassifier(estimators=[('LR', clf1), ('BNB', clf2),
('SVC', clf3)],  voting='hard')
eclf = eclf.fit(xtrain_tfidf, y_train1)
y_pred2 = eclf.predict(xtest_tfidf)
# print(y_pred01)
accuracy3 = accuracy_score(y_test1, y_pred2)

print("Test Accuracy:", round(accuracy3*100, 4))

print("\n", classification_report(y_test1, y_pred2))

Test Accuracy: 65.109

               precision    recall  f1-score   support

         HOF       0.67      0.29      0.40       131
         NOT       0.65      0.90      0.75       190

    accuracy                           0.65       321
   macro avg       0.66      0.60      0.58       321
weighted avg       0.66      0.65      0.61       321



**Evaluation for test set**

In [ ]:
X_train=df1['clean_text']
X_test=df2['clean_text']
y_train=df1['task_1']

In [ ]:
X_train.shape, X_test.shape

((1281,), (320,))

In [ ]:
tfidf_vectorizer11 = TfidfVectorizer(analyzer=custom_analyzer)
tfidf_vectorizer11.fit_transform(X_train)
xtrain_tfidf11 =  tfidf_vectorizer11.transform(X_train)
xtest_tfidf11 =  tfidf_vectorizer11.transform(X_test)

SVM model submission

In [ ]:
svm_model1 = SVC(kernel='linear', class_weight='balanced')

In [ ]:
svm_model1.fit(xtrain_tfidf11, y_train)

SVC(class_weight='balanced', kernel='linear')

In [ ]:
y_pred11 = svm_model1.predict(xtest_tfidf11)

In [ ]:
#Submission
# to create .csv file consisting of tweet ids and predicted labels
y_pred_df= pd.DataFrame(data=y_pred11, columns=['y_pred11'])
Submisssion_bengali = pd.DataFrame()
Submisssion_bengali['S. No.'] = df2['S. No.']
Submisssion_bengali['task_1 '] = y_pred_df
Submisssion_bengali.to_csv('/content/drive/MyDrive/MUCS_test_Bengali_run7.csv',index = None)

Ensembler submission


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

clf11 = LogisticRegression(random_state=1)
clf22 = BernoulliNB()
clf33 = LinearSVC(penalty='l2',C=1.0)

eclf11 = VotingClassifier(estimators=[('LR', clf11), ('BNB', clf22),
('SVC', clf33)],  voting='hard')
eclf11 = eclf11.fit(xtrain_tfidf11, y_train)
y_pred12 = eclf11.predict(xtest_tfidf11)
# # print(y_pred01)
# accuracy3 = accuracy_score(y_test1, y_pred2)

# print("Test Accuracy:", round(accuracy3*100, 4))

# print("\n", classification_report(y_test1, y_pred2))

In [ ]:
#Submission
# to create .csv file consisting of tweet ids and predicted labels
y_pred_df11= pd.DataFrame(data=y_pred12, columns=['y_pred12'])
Submisssion_bengali = pd.DataFrame()
Submisssion_bengali['S. No.'] = df2['S. No.']
Submisssion_bengali['task_1 '] = y_pred_df11
Submisssion_bengali.to_csv('/content/drive/MyDrive/MUCS_test_Bengali_run8.csv',index = None)

Random forest submission

In [ ]:
RFC1 = RandomForestClassifier()

In [ ]:
RFC1.fit(xtrain_tfidf11, y_train)

RandomForestClassifier()

In [ ]:
y_pred13 = RFC1.predict(xtest_tfidf11)

In [ ]:
#Submission
# to create .csv file consisting of tweet ids and predicted labels
y_pred_df12= pd.DataFrame(data=y_pred13, columns=['y_pred13'])
Submisssion_bengali = pd.DataFrame()
Submisssion_bengali['S. No.'] = df2['S. No.']
Submisssion_bengali['task_1 '] = y_pred_df12
Submisssion_bengali.to_csv('/content/drive/MyDrive/MUCS_test_Bengali_run9.csv',index = None)

Logistic regression

In [ ]:
LR=LogisticRegression()

In [ ]:
LR.fit(xtrain_tfidf11, y_train)

LogisticRegression()

In [ ]:
y_pred14 = LR.predict(xtest_tfidf11)

In [ ]:
#Submission
# to create .csv file consisting of tweet ids and predicted labels
y_pred_df13= pd.DataFrame(data=y_pred14, columns=['y_pred14'])
Submisssion_bengali = pd.DataFrame()
Submisssion_bengali['S. No.'] = df2['S. No.']
Submisssion_bengali['task_1 '] = y_pred_df13
Submisssion_bengali.to_csv('/content/drive/MyDrive/MUCS_test_Bengali_run10.csv',index = None)